# 인공 신경망

In [1]:
# 실행마다 동일한 결과를 얻기 위해 케라스에 랜덤 시드를 사용하고 텐서플로 연산을 결정적으로 만든다
import tensorflow as tf

tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

### 패션 MINST

In [2]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## 인공 신경망으로 모델 만들기

In [4]:
# 각 샘플을 1차원 배열로 재배열
# 픽셀은 0~255 사이의 정수값을 가짐 -> 255로 나누어 0~1 사이의 값으로 정규화하는것이 보통임
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)

print(train_scaled.shape)
# 784개의 픽셀로 이루어진 60000개의 샘플

(60000, 784)


In [5]:
from sklearn.model_selection import train_test_split

# train, test 데이터 분할
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [6]:
print(train_scaled.shape, train_target.shape)
print(val_scaled.shape, val_target.shape)

(48000, 784) (48000,)
(12000, 784) (12000,)


- 10개의 패션 아이템을 분류하기 위해 10개의 뉴런으로 분류된다.
- z1 ~ z10

### 밀집층 - 완전 연결층
- 784개의 모든 픽셀과 오른쪽의 뉴런이 모두 연결된 상태


In [11]:
from keras.layers import Input, Dense

# 10개의 뉴런 지정
# 다중 분류를 위해 softmax 함수 사용
# input_shape 지정
dense = keras.layers.Dense(10, activation='softmax', input_shape=(784,))

In [13]:
# 위에서 만든 밀집층을 가진 신경망 모델을 만들기 위해 Sequential 클래스를 이용해 model을 생성한다.
# Sequential 클래스의 매개변수에 dense를 전달
model = keras.Sequential([dense])

# 여기서 만든 model 객체가 바로 신경망 모델이다.
# 절편이 뉴런마다 더해진다는것이 중요

- 소프트맥스와 같이 뉴런의 선형 방정식 계산 결과에 적용되는 함수를 활성화 함수라고 한다.

## 인공 신경망으로 패션 아이템 분류하기
- 손실 함수의 종류 지정하기
  - 이진 분류: binary_crossentropy
  - 다중 분류: categorical_crossentropy
- spars라는 단어가 붙은 이유
  - 하나의 뉴런으로 양성과 음성 클래스에 대한 크로스 엔트로피 손실을 모두 계산하기 위해
   

In [15]:
# 훈련하기 전 compile 함수로 설정단계 진행
# 손실 함수를 지정하고, metrics를 지정한다.
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### 원-핫 인코딩
- 다중 분류에서 크로스 엔트로피 손실 함수를 사용하려면 정수로 된 타깃값을 원-핫 인코딩으로 변환해야된다.
- 손실을 낮추려면 활성화 뉴런의 값을 가능한 1에 가깝게 만들어야함 -> 다른 타겟들과의 연관관계를 제거해야한다.
  - 그러나 sparse_categorical_crossentropy는 정수값 사용 가능
- 티셔츠 샘플의 타깃은 정숫값 0 -> 1000000000
- 바지 샘플의 타깃은 정숫값 1 -> 0100000000

### metrics
- keras는 모델이 훈련할 때 기본적으로 에포크마다 손실 값을 출력해준다.
- 이 때 정확도도 함께 출력을 하기 위해서 'accuracy'를 지정한것이다.

In [17]:
# 모델 훈련
# epoches: 반복횟수
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7335 - loss: 0.7930
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8338 - loss: 0.4857
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8446 - loss: 0.4569
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8503 - loss: 0.4427
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8534 - loss: 0.4339


- 에포크마다 걸린 시간과 손실, 정확도를 출력해준다.

In [19]:
# 검증 세트를 통해 모델의 성능 확인
model.evaluate(val_scaled, val_target)

375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8465 - loss: 0.4372


[0.4451083540916443, 0.847000002861023]